In [244]:
import hiplot as hip
import pandas as pd
import matplotlib.pyplot as plt
import uproot3 as uproot
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
import random
import numpy as np
max=np.max
import seaborn as sns
from sklearn.decomposition import PCA
import os
import sys
from scipy.optimize import curve_fit
from scipy.integrate import quad

import mplhep as hep
mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use([hep.styles.ATLAS])
# mpl.rcParams['mathtext.fontset'] = 'stix'
# mpl.rcParams['font.family'] = 'STIXGeneral'

from IPython.core.display import display, HTML
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
    '</style>'
))

SMALL_SIZE = 20
MEDIUM_SIZE = 22
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=17)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

%matplotlib inline

### Load Barlow CSV

In [252]:
floc='/d/grid17/ln16/dselector_v3/study_pwa/mass_dependent_fits/shared_results/systematic_nominal_v9/systematic_v24/SOURCE/systematic_syst/barlows.csv'

In [253]:
df=pd.read_csv(floc,dtype={'tbin':str})

In [254]:
variations=df.variation.unique()

In [255]:
df['posDiff']=df['posVar']-df['posNom']
df['negDiff']=df['negVar']-df['negNom']

In [256]:
prettyNames={
                'posNom':'Nom.',
                'posDiff':'(+)Diff. [nb]',
                'nBarlowPos':r'(+)$N_{Barlow}$',
                'negNom':'Nom.',
                'negDiff':'(-)Diff. [nb]',
                'nBarlowNeg':r'$(-)N_{Barlow}$',
                'variation':'Variation',
                't':r'$-t~\text{GeV}^2$',
            }

output_df=df[['t','variation','posNom','posDiff','nBarlowPos','negNom','negDiff','nBarlowNeg']]

In [262]:
ts=output_df.t.unique()
threshold=0

tmp=output_df[((output_df.nBarlowPos.abs()>threshold)|(output_df.nBarlowNeg.abs()>threshold))].reset_index(drop=True)
pos=tmp.posNom.unique()[0]
neg=tmp.negNom.unique()[0]
tmp=tmp.drop(['posNom','negNom'],axis=1)

# Clean up the variable names
tmap={k:v for k,v in zip(tmp.t.unique(), [r'$0.1<-t<0.2$',r'$0.2<-t<0.325$',r'$0.325<-t<0.5$',r'$0.5<-t<0.75$',r'$0.75<-t<1.0$'])}
tmp.t=tmp.t.map(tmap)
tmp=tmp.rename(columns=prettyNames)

print(tmp.to_latex(escape=False,index=False))

\begin{tabular}{llrrrr}
\toprule
$-t~\text{GeV}^2$ &                   Variation &  (+)Diff. [nb] &  (+)$N_{Barlow}$ &  (-)Diff. [nb] &  $(-)N_{Barlow}$ \\
\midrule
     $0.1<-t<0.2$ &  $|P_{\gamma}|$ lower limit &        0.00063 &          0.04356 &        0.00047 &          0.04475 \\
     $0.1<-t<0.2$ &  $|P_{\gamma}|$ upper limit &       -0.00043 &          0.03098 &       -0.00016 &          0.01590 \\
     $0.1<-t<0.2$ &      $\phi_{\gamma}$ offset &       -0.00169 &          0.12035 &       -0.00086 &          0.08653 \\
     $0.1<-t<0.2$ &      $1.04<M(\eta\pi)<1.68$ &       -0.00325 &          0.23216 &        0.00413 &          0.35772 \\
     $0.1<-t<0.2$ &      $1.04<M(\eta\pi)<1.64$ &       -0.00008 &          0.00535 &        0.00356 &          0.26227 \\
     $0.1<-t<0.2$ &      $1.04<M(\eta\pi)<1.60$ &       -0.00383 &          0.23984 &        0.00015 &          0.01126 \\
     $0.1<-t<0.2$ &          pcwsBinWidth 20MeV &        0.01460 &          0.97553 &        0.00

In [266]:
df['absDiffPos']=df['diffPos'].abs()
df['absDiffNeg']=df['diffNeg'].abs()

print(df['class'].unique())

for classChoice in df['class'].unique():
    for t in df.tbin.unique():
        significantPos=df[(abs(df['nBarlowPos'])>threshold)&(df['tbin']==t)]
        significantPos=significantPos.loc[significantPos['class']==classChoice,'absDiffPos'].max()
        significantPos=0 if np.isnan(significantPos) else significantPos
        print(f'{significantPos:0.4f}',end=' & ')
    print()
    for t in df.tbin.unique():
        significantNeg=df[(abs(df['nBarlowNeg'])>threshold)&(df['tbin']==t)]
        significantNeg=significantNeg.loc[significantNeg['class']==classChoice,'absDiffNeg'].max()
        significantNeg=0 if np.isnan(significantNeg) else significantNeg
        print(f'{significantNeg:0.4f}',end=' & ')
    print()


['polarization' 'fitrange' 'pcws_width' 'bw_param']
0.0017 & 0.0004 & 0.0001 & 0.0005 & 0.0001 & 
0.0009 & 0.0011 & 0.0004 & 0.0001 & 0.0002 & 
0.0038 & 0.0079 & 0.0008 & 0.0028 & 0.0006 & 
0.0041 & 0.0077 & 0.0029 & 0.0002 & 0.0009 & 
0.0146 & 0.0145 & 0.0054 & 0.0017 & 0.0002 & 
0.0084 & 0.0117 & 0.0019 & 0.0013 & 0.0005 & 
0.0095 & 0.0054 & 0.0015 & 0.0027 & 0.0005 & 
0.0248 & 0.0132 & 0.0020 & 0.0004 & 0.0006 & 


In [229]:
print(significantPosList)

['0.0191', '0.0000', '0.0033', '0.0027', '0.0009']


In [228]:
significantNegList

['0.0344', '0.0024', '0.0000', '0.0000', '0.0000']

In [285]:
df.loc[df.tbin=='010020','posNom'].iloc[0]

0.15835